In [ ]:
import pandas as pd
from emutools.tex import StandardTexDoc
import arviz as az
from inputs.constants import ANALYSIS_START_DATE, ANALYSIS_END_DATE, SUPPLEMENT_PATH
from aust_covid.plotting import plot_key_outputs, plot_cdr_examples, plot_subvariant_props, plot_dispersion_examples
from aust_covid.calibration import get_priors, get_targets

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')

In [ ]:
spaghetti = pd.read_csv('spaghetti.csv', header=[0, 1, 2], index_col=[0])
burnt_idata = az.from_netcdf('idata_for_spaghetti.nc')

In [ ]:
targets = get_targets(app_doc)

In [ ]:
plot_key_outputs(burnt_idata, spaghetti, ANALYSIS_START_DATE, ANALYSIS_END_DATE, app_doc, ['notifications_ma', 'deaths_ma'], targets[0].data, targets[1].data, targets[2].data, show_fig=True)